## Entrenamiento del modelo y predicciones en test

En primer lugar entrenaremos el modelo con tres tipos de facturas y realizaremos predcciones sobre una cuarta.<br> Si bien el modelo final estará entrenado con los cuatro tipos de factura, este paso intermedio nos permite hacernos una idea de qué tal generaliza.

In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install seqeval
!pip install accelerate -U
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict, Features, Sequence, ClassLabel, Value
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, logging
from transformers import __version__ as transformers_version
import numpy as np
from sklearn.model_selection import train_test_split
import seqeval.metrics
from google.colab import files
from google.colab import drive
import evaluate
import numpy as np


# Configurar el nivel de verbosidad del logging
logging.set_verbosity_error()


In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
csv_path = "/content/drive/MyDrive/dataset_facturas_aumentado_etiquetado_test.csv"
df = pd.read_csv(csv_path)

# Verificar que se ha leído correctamente
print("Las primeras filas del DataFrame:")
df.head(3)


Las primeras filas del DataFrame:


,id,texto,ner_tags,json,original,json_comprobacion,json_2
0,factura_264,"['L', '..', '.', 'u', '.', '.', 'n', '..e', '....","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '...",1,"{'nombre_cliente': ['SALUD MONTALVO CUÉLLAR', ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '..."
1,factura_311,"['L', '..', '.', 'u', '.', '.', 'n', '..e', '....","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",{'nombre_cliente': 'Celestino Estrada Santiago...,1,{'nombre_cliente': ['Celestino Estrada Santiag...,{'nombre_cliente': 'Celestino Estrada Santiago...
2,factura_473,"['L', '..', '.', 'u', '.', '.', 'n', '..e', '....","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","{'nombre_cliente': 'Candelaria Ayala Icaza', '...",1,"{'nombre_cliente': ['Candelaria Ayala Icaza', ...","{'nombre_cliente': 'Candelaria Ayala Icaza', '..."


Lo primero que tenemos que hacer es convertir el DataFrame en un objeto datasets de Hugging Face.










In [ ]:
# Seleccionar solo las columnas 'id', 'texto' y 'ner_tags'
df_subset = df.loc[:, ['id', 'texto', 'ner_tags']]

# Renombrar la columna 'texto' a 'tokens'
df_subset.rename(columns={'texto': 'tokens'}, inplace=True)

# Mostrar las primeras filas del nuevo DataFrame para verificar
df_subset.head(3)

,id,tokens,ner_tags
0,factura_264,"['L', '..', '.', 'u', '.', '.', 'n', '..e', '....","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,factura_311,"['L', '..', '.', 'u', '.', '.', 'n', '..e', '....","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,factura_473,"['L', '..', '.', 'u', '.', '.', 'n', '..e', '....","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [ ]:
# Definir una función para convertir el contenido de una columna en una lista
def to_list(column):
    # Eliminar los corchetes adicionales y convertir la cadena en lista
    return eval(column)

# Aplicar la función a las columnas 'tokens' y 'ner_tags'
df_subset['tokens'] = df_subset['tokens'].apply(to_list)
df_subset['ner_tags'] = df_subset['ner_tags'].apply(to_list)

In [ ]:
# Definir la secuencia de palabras que queremos buscar en tokens
sequence_to_find = ['Ideas', 'que', 'facilitan', 'su', 'día']

# Función para encontrar la secuencia en la lista de tokens
def find_sequence(tokens):
    for i in range(len(tokens) - len(sequence_to_find) + 1):
        if tokens[i:i+len(sequence_to_find)] == sequence_to_find:
            return True
    return False

# Filtrar las filas que contienen la secuencia en 'tokens'
test_rows = df_subset[df_subset['tokens'].apply(find_sequence)]

# Crear el DataFrame test con las filas seleccionadas
test = pd.DataFrame(test_rows)

# Eliminar las filas seleccionadas del DataFrame original
df_subset.drop(test_rows.index, inplace=True)

# Verificar el nuevo DataFrame de validación
test.head(3)

,id,tokens,ner_tags
0,factura_264,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,factura_311,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,factura_473,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [ ]:
print(test.info())
print(df_subset.info())

<class 'pandas.core.frame.DataFrame'>
Index: 275 entries, 0 to 274
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        275 non-null    object
 1   tokens    275 non-null    object
 2   ner_tags  275 non-null    object
dtypes: object(3)
memory usage: 8.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1278 entries, 275 to 1552
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1278 non-null   object
 1   tokens    1278 non-null   object
 2   ner_tags  1278 non-null   object
dtypes: object(3)
memory usage: 30.1+ KB
None


In [ ]:
# Dividir df_subset en conjuntos de entrenamiento y validación
train, validation = train_test_split(df_subset, test_size=0.2, random_state=42)

# Imprimir el número de filas en los conjuntos de entrenamiento y prueba
print("Número de filas en el conjunto de entrenamiento:", len(train))
print("Número de filas en el conjunto de validación:", len(validation))

Número de filas en el conjunto de entrenamiento: 1022
Número de filas en el conjunto de validación: 256


In [ ]:
# Definir la lista de etiquetas
label_list = ['O', 'B-NOM', 'I-NOM', 'B-DNI', 'I-DNI', 'B-CAL', 'I-CAL', 'B-CP', 'I-CP', 'B-LOC', 'I-LOC', 'B-PRO', 'I-PRO',
            'B-NOMC', 'I-NOMC', 'B-CIF', 'I-CIF', 'B-DIRC', 'I-DIRC', 'B-CPC', 'I-CPC', 'B-LOCC', 'I-LOCC', 'B-PROC', 'I-PROC',
            'B-NUMF', 'I-NUMF', 'B-INI', 'I-INI', 'B-FIN', 'I-FIN', 'B-FAC', 'I-FAC', 'B-CAR','I-CAR', 'B-PER',
            'I-PER', 'B-POT', 'I-POT']
label_to_id = {label: i for i, label in enumerate(label_list)}

num_labels = len(label_list)

In [ ]:
# Función para convertir etiquetas de texto a etiquetas numéricas
def convert_labels_to_ids(labels, label_to_id):
    return [[label_to_id[label] for label in label_seq] for label_seq in labels]

# Convertir las etiquetas en cada DataFrame
def process_dataframe(df, label_to_id):
    df['ner_tags'] = convert_labels_to_ids(df['ner_tags'], label_to_id)
    return df


test = process_dataframe(test, label_to_id)
validation = process_dataframe(validation, label_to_id)
train = process_dataframe(train, label_to_id)

In [ ]:
# Convertir DataFrames a Datasets de Hugging Face
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)
validation_dataset = Dataset.from_pandas(validation)

# Eliminar la columna '__index_level_0__' si existe
train_dataset = train_dataset.remove_columns(['__index_level_0__'])
test_dataset = test_dataset.remove_columns(['__index_level_0__'])
validation_dataset = validation_dataset.remove_columns(['__index_level_0__'])

# Definir las características del dataset
features = Features({
    'id': Value(dtype='string'),
    'tokens': Sequence(Value(dtype='string')),
    'ner_tags': Sequence(ClassLabel(names=label_list))
})

# Asignar las características a cada dataset
train_dataset = train_dataset.cast(features)
test_dataset = test_dataset.cast(features)
validation_dataset = validation_dataset.cast(features)

# Crear el DatasetDict
datasets = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': validation_dataset
})

print(datasets)


Casting the dataset:   0%|          | 0/1022 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/275 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/256 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1022
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 275
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 256
    })
})


In [ ]:
datasets['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-NOM', 'I-NOM', 'B-DNI', 'I-DNI', 'B-CAL', 'I-CAL', 'B-CP', 'I-CP', 'B-LOC', 'I-LOC', 'B-PRO', 'I-PRO', 'B-NOMC', 'I-NOMC', 'B-CIF', 'I-CIF', 'B-DIRC', 'I-DIRC', 'B-CPC', 'I-CPC', 'B-LOCC', 'I-LOCC', 'B-PROC', 'I-PROC', 'B-NUMF', 'I-NUMF', 'B-INI', 'I-INI', 'B-FIN', 'I-FIN', 'B-FAC', 'I-FAC', 'B-CAR', 'I-CAR', 'B-PER', 'I-PER', 'B-POT', 'I-POT'], id=None), length=-1, id=None)

El modelo que vamos a fine-tunear es 'dccuchile/bert-base-spanish-wwm-cased', conocido coloquialmente como BERTO.








In [ ]:
task = "ner"
#model_checkpoint = "mrm8488/distilbert-base-multi-cased-finetuned-typo-detection"
model_checkpoint =  'dccuchile/bert-base-spanish-wwm-cased'
batch_size = 16

Una vez creado el objeto datasets y cargado el modelo, procedemos a tokenizar el texto y alinear las etiquetas NER para que siga habiendo correspondencia.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [ ]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    # Tokenizar las entradas con truncamiento
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Los tokens especiales tienen un ID de palabra que es None. Se establece la etiqueta en -100 para que se ignoren en la función de pérdida.
            if word_idx is None:
                label_ids.append(-100)
            # Se establece la etiqueta para el primer token de cada palabra.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # Para los otros tokens en una palabra, se establece la etiqueta en la etiqueta actual o -100, dependiendo del flag label_all_tokens.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1022 [00:00<?, ? examples/s]

Map:   0%|          | 0/275 [00:00<?, ? examples/s]

Map:   0%|          | 0/256 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1022
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 275
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 256
    })
})

Ahora procedemos a entrenar y evaluar el modelo.












In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    ignore_mismatched_sizes=True
)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate= 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=12,
    weight_decay=0.35,
    push_to_hub=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Definir el data collator
data_collator = DataCollatorForTokenClassification(tokenizer)


In [ ]:
# Cargar la métrica utilizando evaluate
metric = evaluate.load("seqeval")


In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()


{'loss': 1.9422, 'grad_norm': 1.9087363481521606, 'learning_rate': 1.9739583333333333e-05, 'epoch': 0.15625}
{'loss': 0.7462, 'grad_norm': 0.7437829375267029, 'learning_rate': 1.9479166666666668e-05, 'epoch': 0.3125}
{'loss': 0.6056, 'grad_norm': 0.9398928880691528, 'learning_rate': 1.9218750000000003e-05, 'epoch': 0.46875}
{'loss': 0.4498, 'grad_norm': 0.8951431512832642, 'learning_rate': 1.8958333333333334e-05, 'epoch': 0.625}
{'loss': 0.327, 'grad_norm': 1.347677230834961, 'learning_rate': 1.869791666666667e-05, 'epoch': 0.78125}
{'loss': 0.2311, 'grad_norm': 0.7229127287864685, 'learning_rate': 1.84375e-05, 'epoch': 0.9375}
{'eval_loss': 0.15326812863349915, 'eval_precision': 0.750110180696342, 'eval_recall': 0.7670121676430824, 'eval_f1': 0.7584670231729056, 'eval_accuracy': 0.9647058823529412, 'eval_runtime': 10.3454, 'eval_samples_per_second': 24.745, 'eval_steps_per_second': 1.547, 'epoch': 1.0}
{'loss': 0.1704, 'grad_norm': 0.5459885597229004, 'learning_rate': 1.81770833333333

TrainOutput(global_step=768, training_loss=0.08087269943886592, metrics={'train_runtime': 1377.5295, 'train_samples_per_second': 8.903, 'train_steps_per_second': 0.558, 'train_loss': 0.08087269943886592, 'epoch': 12.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.03651632368564606, 'eval_precision': 0.9418707845225417, 'eval_recall': 0.9565570076611086, 'eval_f1': 0.9491570898358896, 'eval_accuracy': 0.9911228553921568, 'eval_runtime': 10.7999, 'eval_samples_per_second': 23.704, 'eval_steps_per_second': 1.481, 'epoch': 12.0}


{'eval_loss': 0.03651632368564606,
 'eval_precision': 0.9418707845225417,
 'eval_recall': 0.9565570076611086,
 'eval_f1': 0.9491570898358896,
 'eval_accuracy': 0.9911228553921568,
 'eval_runtime': 10.7999,
 'eval_samples_per_second': 23.704,
 'eval_steps_per_second': 1.481,
 'epoch': 12.0}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'CAL': {'precision': 0.9473684210526315,
  'recall': 0.9629629629629629,
  'f1': 0.9551020408163265,
  'number': 243},
 'CAR': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 277},
 'CIF': {'precision': 0.9929078014184397,
  'recall': 0.9960474308300395,
  'f1': 0.9944751381215469,
  'number': 1265},
 'CP': {'precision': 0.9875666074600356,
  'recall': 0.984070796460177,
  'f1': 0.9858156028368794,
  'number': 565},
 'CPC': {'precision': 0.9892328398384926,
  'recall': 1.0,
  'f1': 0.9945872801082544,
  'number': 735},
 'DIRC': {'precision': 0.9144295302013423,
  'recall': 0.8934426229508197,
  'f1': 0.9038142620232172,
  'number': 610},
 'DNI': {'precision': 0.9878787878787879,
  'recall': 1.0,
  'f1': 0.9939024390243903,
  'number': 326},
 'FAC': {'precision': 0.6989898989898989,
  'recall': 0.7972350230414746,
  'f1': 0.7448869752421959,
  'number': 434},
 'FIN': {'precision': 0.9306167400881057,
  'recall': 0.95372460496614,
  'f1': 0.9420289855072463,
  'number': 886},
 'I

In [ ]:
# Carga los datos de prueba
test_dataset = tokenized_datasets["test"]

# Evalúa el modelo en el conjunto de datos de prueba
results = trainer.evaluate(test_dataset)

# Imprime los resultados
print(results)

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.03982982411980629, 'eval_precision': 0.9363899215200331, 'eval_recall': 0.9440326476222204, 'eval_f1': 0.9401957531519575, 'eval_accuracy': 0.9909732620320856, 'eval_runtime': 11.7729, 'eval_samples_per_second': 23.359, 'eval_steps_per_second': 1.529, 'epoch': 12.0}
{'eval_loss': 0.03982982411980629, 'eval_precision': 0.9363899215200331, 'eval_recall': 0.9440326476222204, 'eval_f1': 0.9401957531519575, 'eval_accuracy': 0.9909732620320856, 'eval_runtime': 11.7729, 'eval_samples_per_second': 23.359, 'eval_steps_per_second': 1.529, 'epoch': 12.0}


In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'CAL': {'precision': 0.9637681159420289,
  'recall': 0.982274741506647,
  'f1': 0.9729334308705193,
  'number': 677},
 'CAR': {'precision': 0.10344827586206896,
  'recall': 0.02247191011235955,
  'f1': 0.036923076923076927,
  'number': 267},
 'CP': {'precision': 0.988939492517892,
  'recall': 0.9819121447028424,
  'f1': 0.9854132901134521,
  'number': 1548},
 'CPC': {'precision': 0.8822714681440443,
  'recall': 0.9984326018808778,
  'f1': 0.9367647058823529,
  'number': 638},
 'DIRC': {'precision': 0.855457227138643,
  'recall': 0.9105180533751962,
  'f1': 0.8821292775665399,
  'number': 637},
 'DNI': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1495},
 'FIN': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'LOC': {'precision': 0.9571209800918836,
  'recall': 0.9674922600619195,
  'f1': 0.962278675904542,
  'number': 1292},
 'LOCC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'NOM': {'precision': 0.9932432432432432,
  'recall': 0.984375,
  'f1': 0

Si bien las métricas pueden parecer muy buenas, al haber tantas clases y estar estas tan desbalanceadas, el resultado es engañoso.






